In [1]:
import torch
from tqdm import tqdm
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import torch.optim as optim
from utils import sample_along_rays,volume_rendering,psnr,intrinsic_matrix
from eval import render_images
from dataset import load_data_numpy,RaysData
from model import NeRF_Base
from train import train


d:\Anaconda3\envs\built-nerf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
images_train, c2ws_train, images_val, c2ws_val, c2ws_test, focal = load_data_numpy('lego_200x200.npz')

In [4]:
images_train = torch.tensor(images_train).float().to(device)
c2ws_train = torch.tensor(c2ws_train).float().to(device)
images_val = torch.tensor(images_val).float().to(device)
c2ws_val = torch.tensor(c2ws_val).float().to(device)
c2ws_test = torch.tensor(c2ws_test).float().to(device)
focal = torch.tensor(focal).float().to(device)
K_train = intrinsic_matrix(focal.item(), focal.item(), images_train.shape[1] / 2, images_train.shape[2] / 2).unsqueeze(0).repeat(images_train.shape[0], 1, 1).to(device)
K_val = intrinsic_matrix(focal.item(), focal.item(), images_val.shape[1] / 2, images_val.shape[2] / 2).unsqueeze(0).repeat(images_val.shape[0], 1, 1).to(device)
K_test = intrinsic_matrix(focal.item(), focal.item(), images_val.shape[1] / 2, images_val.shape[2] / 2).unsqueeze(0).repeat(c2ws_test.shape[0], 1, 1).to(device)

In [5]:
model = NeRF_Base().to(device)
optimizer = optim.Adam(model.parameters(),lr= 10e-4)
criterion = nn.MSELoss()
batch_size = 32
num_epochs = 100
train_loader = RaysData(images_train,K_train,c2ws_train)
val_loader = RaysData(images_val, K_val, c2ws_val)
test_loader = RaysData(images_train[:60], K_test, c2ws_test)
train(model, train_loader, val_loader, test_loader, optimizer, criterion, num_epochs=num_epochs, batch_size=batch_size, device=device)

Training PSNR: 13.859389050796487
Training PSNR: 11.375910751987515
Training PSNR: 12.46124462978422
Training PSNR: 13.913269686306187
Training PSNR: 11.785611181498478
Training PSNR: 12.660786205931272
Training PSNR: 11.381140199323887
Training PSNR: 9.79670300814669
Training PSNR: 10.994457014286795
Training PSNR: 12.570654943404351
Training PSNR: 12.538048331118647
Training PSNR: 14.067156199951476
Training PSNR: 11.155669518189024
Training PSNR: 9.901007258811699
Training PSNR: 13.237616483723922
Training PSNR: 13.037226293572887
Training PSNR: 10.956436605660404
Training PSNR: 12.92935251350002
Training PSNR: 12.381255814561008
Training PSNR: 9.708685596918151
Training PSNR: 13.623684050933866
Training PSNR: 12.26391968600324
Training PSNR: 12.478711316942983
Training PSNR: 10.517049380141476
Training PSNR: 13.430530180697684


RuntimeError: CUDA out of memory. Tried to allocate 2.44 GiB (GPU 0; 3.00 GiB total capacity; 3.72 GiB already allocated; 0 bytes free; 3.80 GiB reserved in total by PyTorch)

In [ ]:
a = torch.randn(64).unsqueeze(-1).unsqueeze(-1)
print(a.shape)
b = torch.randn((32,3))
print(b.shape)
c = a*b
print(c.shape)

torch.Size([64, 1, 1])
torch.Size([32, 3])
torch.Size([64, 32, 3])


In [ ]:
a = torch.Tensor([1,2,3])
b = torch.Tensor([4,5,6])
c = torch.stack((a,b),dim=-1)
c

tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])